In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sqlalchemy import create_engine
from sqlalchemy import MetaData


In [2]:
#Loading Database credentials 
username = 'postgres'
password = 'myproject'
hostname = 'database-1.cfg4ma0mq56c.us-east-2.rds.amazonaws.com'
port = '5432'
database_name = 'tx-dx'

# Create connection string
engine = create_engine(f'postgresql://{username}:{password}@{hostname}:{port}/{database_name}')

In [3]:
#Testing connection to database server 
try:
    connection = engine.connect()
    print("Connection successful!")
    connection.close()
except Exception as e:
    print(f"Connection failed with error: {e}")

Connection successful!


In [4]:
#Querrying our table to ensure everything is running smoothly 
sql_query = "SELECT * FROM claim"
result = engine.execute(sql_query)

In [5]:
# Fetch all rows from the result
rows = result.fetchall()

In [6]:
#Fetching all coumn names 
columns = result.keys()

In [7]:
#Loading Dataframe and creating a copy
df = pd.DataFrame(rows, columns=columns)
data = df.copy()

In [8]:
connection.close()

In [9]:
#Loading out data 
data.head()

,index,Patient,Age,Age_Group,Sex,Diagnosis_Code,Diagnosis_Group,Diagnosis_Family,Diagnosis_Description,Med_Code,Med_Description,Med_Description_Simp,Quantity,Status,Amount_Billed,Amount_Paid
0,0,2112140237,37,26-45,Male,K21.9,K2,K,Gastro-esophageal reflux disease without esoph...,1.738110e+13,(SODIUM CHLORIDE : 9 MG/ML) SOLUTION FOR INFU...,SODIUM CHLORIDE,1,Paid,3.00,3.00
1,1,2002110188,38,26-45,Male,I21.3,I0,I,ST elevation (STEMI) myocardial infarction of ...,9.933860e+12,(CLOPIDOGREL (AS BESILATE) : 75 MG) FILM COAT...,CLOPIDOGREL,4,Rejected,17.44,0.00
2,2,1510110229,59,46-65,Male,B34.2,B3,B,"Coronavirus infection, unspecified",1.372430e+12,(PANTOPRAZOLE (AS SODIUM) : 40 MG) ENTERIC CO...,PANTOPRAZOLE,2,Paid,5.36,5.36
3,3,2312040128,38,26-45,Male,I69.354,I1,I,Hemiplegia and hemiparesis following cerebral ...,2.717920e+11,(AMLODIPINE : 5 MG) (VALSARTAN : 160 MG) FILM...,"AMLODIPINE, VALSARTAN",7,Paid,51.66,51.66
4,4,2311110151,44,26-45,Male,J32.9,J3,J,"Chronic sinusitis, unspecified",3.551200e+12,(GENTAMICIN : 0.3%) EYE OINTMENT,GENTAMICIN,1,Paid,8.50,8.50


In [10]:

# Select relevant columns for prediction
features = ['Age', 'Sex', 'Diagnosis_Group']
target = 'Status'

X = data[features]
y = data[target]


In [11]:

# Perform one-hot encoding for categorical variables
encoder = OneHotEncoder(drop='first')
X_encoded = encoder.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)


In [12]:

# Initialize and train the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = rf_classifier.predict(X_test)


In [13]:

# Evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.8151283895061585


In [14]:
# Generate classification report
class_report = classification_report(y_test, y_pred)
print("Classification Report:\n", class_report)

Classification Report:
               precision    recall  f1-score   support

        Paid       0.83      0.94      0.88     31164
    Rejected       0.76      0.49      0.60     11947

    accuracy                           0.82     43111
   macro avg       0.79      0.72      0.74     43111
weighted avg       0.81      0.82      0.80     43111

